In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from requests_html import HTMLSession

import time

In [2]:
driver = webdriver.Chrome(executable_path='C:/Users/17178/Downloads/chromedriver_win32/chromedriver.exe')
driver.get(
    'https://www.berkshirehathawayautomotive.com/used-inventory/index.htm?normalBodyStyle=SUV&compositeType=used&start=0')
content = driver.page_source
soup = BeautifulSoup(content)

In [3]:
counter = []

for element in soup.findAll('span', {'class': 'd-none d-sm-inline'}):
    counter.append([int(s) for s in element.contents[0] if s.isdigit()])
    
def convert(counter):
      
    # Converting integer list to string list
    s = [str(i) for i in counter[0]]
      
    # Join list items using join()
    res = int("".join(s))
      
    return(res)

#number of used cars for sale
counter = convert(counter)
counter

3403

In [ ]:
for i in range(0,counter+1,18):
    driver.get(
        f'https://www.berkshirehathawayautomotive.com/used-inventory/index.htm?normalBodyStyle=SUV&compositeType=used&start={i}')
    content = driver.page_source
    soup = BeautifulSoup(content)

    results_year_make = []
    results_miles  = []
    results_price = []
    results_state = []
    results_source = []

    for element in soup.findAll('div', {'class': 'ddc-wrapper'}):
        for year_make in element.findAll('span', {'class': 'ddc-font-size-small'}):
            results_year_make.append(year_make.contents[0])
        for price in element.findAll('span', {'class': 'text-right portal-price'}):
            try:
                results_price.append(price.contents[0])
            except:
                results_price.append(np.nan)
            results_source = 'berkshire'
        for miles in element.findAll('li', {'class': 'odometer'}):
            results_miles.append(miles.contents[0])
        for state in element.findAll('li', {'class': 'accountName'}):
            results_state.append(state.contents[0])
        
        df_temp = pd.DataFrame(
                    {'year_make': results_year_make,
                     'model_trim': 'missing',
                     'price': results_price,
                     'state': results_state,
                     'miles': results_miles,
                     'source': results_source
                    })


        df_temp['miles'] = df_temp['miles'].replace({'\,':''}, regex=True)
        try:
            df_temp['miles'] = df_temp['miles'].str.extract('([0-9]+)').astype(float)
        except:
            df_temp['miles'] = 'missing'

#         df_temp['price'] = df_temp['price'].replace({'\$':''}, regex=True)
#         df_temp['price'] = df_temp['price'].replace({',':''}, regex=True)

        if i == 0:
            df = df_temp
        else:
            df = pd.merge(df, df_temp, how='outer')
            del df_temp

df.to_csv(f'C:/Users/17178/Documents/DS Projects/Nitorum Case/Data/berkshire_suv.csv')
df

In [ ]:
df.isnull().sum()